In [1]:
import os
import sys
import warnings

# Suppress specific warnings
warnings.filterwarnings("ignore", message="This sequence already has </s>.")

# Append path for module imports
scripts_path = os.path.abspath(os.path.join('..', 'scripts'))
sys.path.append(scripts_path)


# Standard library imports
import random
import string

# Third-party imports
import json
import numpy as np
import pandas as pd
import itertools
import pke
import torch
import nltk
from dateutil.parser import parse
from nltk.corpus import stopwords, wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
# from textdistance import levenshtein
from rapidfuzz.distance import Levenshtein as levenshtein

# Download necessary NLTK data
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('brown')
nltk.download('wordnet')

from typing import List, Dict
import re


[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\miiciii\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\miiciii\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\miiciii\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\miiciii\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\miiciii\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# Import custom models from model_manager
from model_manager import (
    get_answergeneration_model,
    get_questiongeneration_model,
    get_sense2vec_model,
    get_sentence_transformer_model,
    get_random_passage
)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
d:\01SetupFiles\AnacondaNavigator\anacondanavigatorfiles\envs\Thesis\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
d:\01SetupFiles\AnacondaNavigator\anacondanavigatorfiles\envs\Thesis\Lib\site-packages\sentence_trans

In [3]:
# Initialize models
t5ag_model, t5ag_tokenizer = get_answergeneration_model()
t5qg_model, t5qg_tokenizer = get_questiongeneration_model()
s2v = get_sense2vec_model()
sentence_transformer_model = get_sentence_transformer_model()
random_passage = get_random_passage()

In [4]:
def answer_question(question, context):
    """Generate an answer for a given question and context."""
    input_text = f"question: {question} context: {context}"
    input_ids = t5ag_tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)
    
    with torch.no_grad():
        output = t5ag_model.generate(input_ids, max_length=512, num_return_sequences=1, max_new_tokens=200)

    return t5ag_tokenizer.decode(output[0], skip_special_tokens=True).capitalize()

def get_passage(passage):
    """Generate a random context from the dataset."""
    return passage.sample(n=1)['context'].values[0]

def get_question(context, answer, model, tokenizer):
    """Generate a question for the given answer and context."""
    answer_span = context.replace(answer, f"<hl>{answer}<hl>", 1) + "</s>"
    inputs = tokenizer(answer_span, return_tensors="pt")
    question = model.generate(input_ids=inputs.input_ids, max_length=50)[0]

    return tokenizer.decode(question, skip_special_tokens=True)

def get_nouns_multipartite(content):
    """Extract keywords from content using MultipartiteRank algorithm."""
    try:
        extractor = pke.unsupervised.MultipartiteRank() # type: ignore
        extractor.load_document(input=content, language='en')
        pos_tags = {'PROPN', 'NOUN', 'ADJ', 'VERB', 'ADP', 'ADV', 'DET', 'CONJ', 'NUM', 'PRON', 'X'}

        stoplist = list(string.punctuation) + ['-lrb-', '-rrb-', '-lcb-', '-rcb-', '-lsb-', '-rsb-']
        stoplist += stopwords.words('english')

        extractor.candidate_selection(pos=pos_tags)
        extractor.candidate_weighting(alpha=1.1, threshold=0.75, method='average')
        keyphrases = extractor.get_n_best(n=15)
        
        return [val[0] for val in keyphrases]
    except Exception as e:
        print(f"Error extracting nouns: {e}")
        return []

def get_keywords(passage):
    """Extract keywords using TF-IDF."""
    try:
        vectorizer = TfidfVectorizer(stop_words='english')
        tfidf_matrix = vectorizer.fit_transform([passage])
        feature_names = vectorizer.get_feature_names_out()
        tfidf_scores = tfidf_matrix.toarray().flatten() # type: ignore
        word_scores = dict(zip(feature_names, tfidf_scores))
        sorted_words = sorted(word_scores.items(), key=lambda x: x[1], reverse=True)
        keywords = [word for word, score in sorted_words]
        return keywords
    except Exception as e:
        print(f"Error extracting keywords: {e}")
        return []

def filter_same_sense_words(original, wordlist):
    """Filter words that have the same sense as the original word."""
    base_sense = original.split('|')[1]
    return [word[0].split('|')[0].replace("_", " ").title().strip() for word in wordlist if word[0].split('|')[1] == base_sense]

def get_max_similarity_score(wordlist, word):
    """Get the maximum similarity score between the word and a list of words."""
    return max(levenshtein.normalized_similarity(word.lower(), each.lower()) for each in wordlist)

def sense2vec_get_words(word, s2v, topn, question):
    """Get similar words using Sense2Vec."""
    try:
        sense = s2v.get_best_sense(word, senses=["NOUN", "PERSON", "PRODUCT", "LOC", "ORG", "EVENT", "NORP", "WORK_OF_ART", "FAC", "GPE", "NUM", "FACILITY"])
        
        if sense is None:
            print(f"[DEBUG] No suitable sense found for word: '{word}'")
            return []

        most_similar = s2v.most_similar(sense, n=topn)
        output = filter_same_sense_words(sense, most_similar)
        
        threshold = 0.6
        final = [word]
        checklist = question.split()

        for similar_word in output:
            if get_max_similarity_score(final, similar_word) < threshold and similar_word not in final and similar_word not in checklist:
                final.append(similar_word)
    
        return final[1:]
    
    except Exception as e:
        print(f"Error in Sense2Vec: {e}")
        return []

def mmr(doc_embedding, word_embeddings, words, top_n, lambda_param):
    """Maximal Marginal Relevance (MMR) for keyword extraction."""
    try:
        word_doc_similarity = cosine_similarity(word_embeddings, doc_embedding)
        word_similarity = cosine_similarity(word_embeddings)

        keywords_idx = [np.argmax(word_doc_similarity)]
        candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

        for _ in range(top_n - 1):
            candidate_similarities = word_doc_similarity[candidates_idx, :]
            target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

            mmr = (lambda_param * candidate_similarities) - ((1 - lambda_param) * target_similarities.reshape(-1, 1))
            mmr_idx = candidates_idx[np.argmax(mmr)]

            keywords_idx.append(mmr_idx) # type: ignore
            candidates_idx.remove(mmr_idx)

        return [words[idx] for idx in keywords_idx]
    except Exception as e:
        print(f"Error in MMR: {e}")
        return []
    
def classify_question_type(question: str) -> str:
    """
    Classify the type of question as literal, evaluative, or inferential.
    
    Parameters:
        question (str): The question to classify.
        
    Returns:
        str: The type of the question ('literal', 'evaluative', or 'inferential').
    """
    # Define keywords or patterns for each question type
    literal_keywords = [
    'what', 'when', 'where', 'who', 'how many', 'how much', 
    'which', 'name', 'list', 'identify', 'define', 'describe', 
    'state', 'mention'
    ]

    evaluative_keywords = [
    'evaluate', 'justify', 'explain why', 'assess', 'critique', 
    'discuss', 'judge', 'opinion', 'argue', 'agree or disagree', 
    'defend', 'support your answer', 'weigh the pros and cons', 
    'compare', 'contrast'
    ]

    inferential_keywords = [
    'why', 'how', 'what if', 'predict', 'suggest', 'imply', 
    'conclude', 'infer', 'reason', 'what might', 'what could', 
    'what would happen if', 'speculate', 'deduce', 'interpret', 
    'hypothesize', 'assume'
    ]


    question_lower = question.lower()
    
    # Check for literal question keywords
    if any(keyword in question_lower for keyword in literal_keywords):
        return 'literal'
    
    # Check for evaluative question keywords
    if any(keyword in question_lower for keyword in evaluative_keywords):
        return 'evaluative'
    
    # Check for inferential question keywords
    if any(keyword in question_lower for keyword in inferential_keywords):
        return 'inferential'
    
    # Default to 'unknown' if no pattern matches
    return 'unknown'

def format_phrase(phrase):
    """Format phrases by replacing spaces with underscores and adding default |n."""
    return phrase.replace(" ", "_") + "|n"

def get_distractors(input_phrases, topn=5):
    """Find similar keywords for a list of input phrases using Sense2Vec and WordNet."""
    result_list = []

    for phrase in input_phrases:
        formatted_phrase = format_phrase(phrase)

        # Check if the phrase exists in the Sense2Vec model
        if formatted_phrase in s2v:
            # Get similar phrases from Sense2Vec
            similar_phrases = s2v.most_similar(formatted_phrase, n=topn)
            similar_keywords = [item[0].split("|")[0].replace("_", " ") for item in similar_phrases]
        else:
            # List similar keys that might exist in the model for exploration
            print(f"'{formatted_phrase}' not found in the model. Exploring similar available keys...")
            available_keys = [key for key in s2v.keys() if phrase.split()[0] in key or phrase.split()[-1] in key]
            print(f"Available keys related to '{phrase}': {available_keys}")

            # Use WordNet to find synonyms if available keys are empty
            if not available_keys:
                print(f"No close match in the model for '{phrase}'. Trying WordNet for synonyms...")
                synonyms = set()
                for syn in wn.synsets(phrase.replace(" ", "_")):
                    for lemma in syn.lemmas(): # type: ignore
                        synonyms.add(lemma.name().replace("_", " "))
                similar_keywords = list(synonyms)[:topn] if synonyms else ["No match found"]
            else:
                # Provide available keys as similar suggestions
                similar_keywords = [key.split("|")[0].replace("_", " ") for key in available_keys[:topn]]

        result_list.append({
            "phrase": phrase,
            "similar_keywords": similar_keywords
        })

    return result_list

def extract_similar_keywords(input_phrases, topn=5):
    """Call get_distractors and extract only the similar_keywords values."""
    # Call the get_distractors function
    distractors_result = get_distractors(input_phrases, topn)
    
    # Extract similar_keywords from the result
    similar_keywords_list = [result["similar_keywords"] for result in distractors_result]
    
    return similar_keywords_list



In [13]:
def get_mca_questions(context, qg_model, qg_tokenizer, sentence_transformer_model, num_questions=5, max_attempts=2) -> List[Dict]:
    """
    Generate multiple-choice questions for a given context.
    
    Parameters:
        context (str): The context from which questions are generated.
        qg_model (T5ForConditionalGeneration): The question generation model.
        qg_tokenizer (T5Tokenizer): The tokenizer for the question generation model.
        s2v (Sense2Vec): The Sense2Vec model for finding similar words.
        sentence_transformer_model (SentenceTransformer): The sentence transformer model for embeddings.
        num_questions (int): The number of questions to generate.
        max_attempts (int): The maximum number of attempts to generate questions.
    
    Returns:
        list: A list of dictionaries with questions and their corresponding distractors.
    """
    output_list = []

    imp_keywords = get_keywords(context)  # Extract keywords only once
    print(f"[DEBUG] Length: {len(imp_keywords)}, Extracted keywords: {imp_keywords}")

    generated_questions = set()
    generated_answers = set()  # Track generated answers
    attempts = 0

    while len(output_list) < num_questions and attempts < max_attempts:
        attempts += 1

        for keyword in imp_keywords:
            if len(output_list) >= num_questions:
                break
            
            question = get_question(context, keyword, qg_model, qg_tokenizer)
            print(f"[DEBUG] Generated question: '{question}' for keyword: '{keyword}'")
            
            # Encode the new question
            new_question_embedding = sentence_transformer_model.encode(question, convert_to_tensor=True)
            is_similar = False

            # Check similarity with existing questions
            for generated_q in generated_questions:
                existing_question_embedding = sentence_transformer_model.encode(generated_q, convert_to_tensor=True)
                similarity = cosine_similarity(new_question_embedding.unsqueeze(0), existing_question_embedding.unsqueeze(0))[0][0]

                if similarity > 0.8:
                    is_similar = True
                    print(f"[DEBUG] Question '{question}' is too similar to an existing question, skipping.")
                    break

            if is_similar:
                continue

            # Generate and check answer
            t5_answer = answer_question(question, context)
            print(f"[DEBUG] Generated answer: '{t5_answer}' for question: '{question}'")
            
            # Skip answers longer than 3 words
            if len(t5_answer.split()) > 3:
                print(f"[DEBUG] Answer '{t5_answer}' is too long, skipping.")
                continue

            if t5_answer in generated_answers:
                print(f"[DEBUG] Answer '{t5_answer}' has already been generated, skipping question.")
                continue

            generated_questions.add(question)
            generated_answers.add(t5_answer)

            # Generate distractors
            distractors = extract_similar_keywords([t5_answer], topn=5)[0]  # Ensure this returns a list
            print(f"list of distractors : {distractors}")
            print(f"length of distractors {len(distractors)}")
            print(f"type : {type(distractors)}")

            # If no distractors were found, use important keywords as distractors
            if not distractors or len(distractors) == 0:
                print("[DEBUG] No distractors found, using important keywords as distractors.")
                distractors = imp_keywords
            else:
                # If answer is a phrase, handle distractors appropriately
                answer_words = t5_answer.split()
                distractors = [d for d in distractors if not any(word.lower() in t5_answer.lower() for word in answer_words)]
                distractors = [d.capitalize() for d in distractors if d.lower() != keyword.lower()]

                if len(distractors) < 3:
                    additional_distractors = [kw.capitalize() for kw in imp_keywords if kw.capitalize() not in distractors and kw.lower() != keyword.lower()]
                    distractors.extend(additional_distractors[:3 - len(distractors)])
                else:
                    distractors = distractors[:3]

            print(f"[DEBUG] Final distractors: {distractors} for question: '{question}'")

            choices = distractors + [t5_answer]
            choices = [item.title() for item in choices]
            random.shuffle(choices)
            print(f"[DEBUG] Options: {choices} for question: '{question}'")

            # Classify question type
            question_type = classify_question_type(question)
            
            output_list.append({
                'answer': t5_answer,
                'answer_length': len(t5_answer),
                'choices': choices,
                'passage': context,
                'passage_length': len(context),
                'question': question,
                'question_length': len(question),
                'question_type': question_type
            })

        print(f"[DEBUG] Generated {len(output_list)} questions so far after {attempts} attempts")

    return output_list[:num_questions]

In [14]:
original_context = get_passage(random_passage)

print(original_context)

questions_and_distractors = get_mca_questions(original_context, t5qg_model, t5qg_tokenizer, sentence_transformer_model, num_questions=5)

OUTPUT_FILE = os.path.join('..', 'outputs', 'plots', 'predictions', 'generated_questions.json')

def convert_to_serializable(data):
    if isinstance(data, pd.Series):
        return data.tolist()
    elif isinstance(data, dict):
        return {k: convert_to_serializable(v) for k, v in data.items()}
    elif isinstance(data, list):
        return [convert_to_serializable(i) for i in data]
    else:
        return data

try:
    with open(OUTPUT_FILE, 'r') as f:
        existing_data = json.load(f)
        if not isinstance(existing_data, list):  # Ensure existing data is a list
            raise ValueError("The existing data is not a list.")
except FileNotFoundError:
    existing_data = []
except json.JSONDecodeError:
    print(f"Warning: {OUTPUT_FILE} contains invalid JSON. Initializing with an empty list.")
    existing_data = []
except ValueError as e:
    print(f"ValueError: {e}")
    existing_data = []
except Exception as e:
    print(f"An unexpected error occurred while reading {OUTPUT_FILE}: {e}")
    existing_data = []

# Convert any non-serializable data to a serializable format
questions_and_distractors_serializable = convert_to_serializable(questions_and_distractors)
existing_data.extend(questions_and_distractors_serializable)

# Write updated data back to the file
try:
    with open(OUTPUT_FILE, 'w') as f:
        json.dump(existing_data, f, indent=4)
    print(f"Data successfully written to {OUTPUT_FILE}.")
except Exception as e:
    print(f"An error occurred while writing to {OUTPUT_FILE}: {e}")

The roots of Greek success in the Ottoman Empire can be traced to the Greek tradition of education and commerce. It was the wealth of the extensive merchant class that provided the material basis for the intellectual revival that was the prominent feature of Greek life in the half century and more leading to the outbreak of the Greek War of Independence in 1821. Not coincidentally, on the eve of 1821, the three most important centres of Greek learning were situated in Chios, Smyrna and Aivali, all three major centres of Greek commerce. Greek success was also favoured by Greek domination of the Christian Orthodox church.
[DEBUG] Length: 43, Extracted keywords: ['greek', '1821', 'centres', 'commerce', 'success', 'aivali', 'basis', 'century', 'chios', 'christian', 'church', 'class', 'coincidentally', 'domination', 'education', 'empire', 'eve', 'extensive', 'favoured', 'feature', 'half', 'important', 'independence', 'intellectual', 'leading', 'learning', 'life', 'major', 'material', 'merch

Both `max_new_tokens` (=200) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[DEBUG] Generated question: 'What was the name of the three most important centres of Greek commerce?' for keyword: 'greek'
[DEBUG] Generated answer: 'Chios, smyrna and aivali' for question: 'What was the name of the three most important centres of Greek commerce?'
[DEBUG] Answer 'Chios, smyrna and aivali' is too long, skipping.


Both `max_new_tokens` (=200) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[DEBUG] Generated question: 'When did the Greek War of Independence begin?' for keyword: '1821'
[DEBUG] Generated answer: '1821' for question: 'When did the Greek War of Independence begin?'
'1821|n' not found in the model. Exploring similar available keys...
Available keys related to '1821': ['__^Ɍ1821|X', '1821|DATE', '1821|NUM']
list of distractors : ['  ^Ɍ1821', '1821', '1821']
length of distractors 3
type : <class 'list'>
[DEBUG] Final distractors: ['Greek', 'Centres', 'Commerce'] for question: 'When did the Greek War of Independence begin?'
[DEBUG] Options: ['Centres', 'Greek', 'Commerce', '1821'] for question: 'When did the Greek War of Independence begin?'


...

[DEBUG] Generated answer: 'Greek domination of the christian orthodox church' for question: 'On what day of 1821 were the three most important centres of Greek learning located?'
[DEBUG] Answer 'Greek domination of the christian orthodox church' is too long, skipping.
[DEBUG] Generated question: 'What type of merchant class provided the material basis for the intellectual revival?' for keyword: 'extensive'


Both `max_new_tokens` (=200) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[DEBUG] Generated answer: 'Extensive merchant class' for question: 'What type of merchant class provided the material basis for the intellectual revival?'
'Extensive_merchant_class|n' not found in the model. Exploring similar available keys...
Available keys related to 'Extensive merchant class': ['future_classes|NOUN', 'classical_definition|NOUN', 'top_classes|NOUN', 'missed_classes|NOUN', 'in-class|NOUN', 'only_two_classes|NOUN', 'classical_Arabic|LANGUAGE', 'fewer_classes|NOUN', 'classical_mechanics|NOUN', 'classic_costume|NOUN', 'fun_classes|NOUN', 'classic_combination|NOUN', 'classified_materials|NOUN', 'class_sizes|NOUN', 'classic_rock_bands|NOUN', 'classic_albums|NOUN', 'Paladin_class|NOUN', 'only_other_class|NOUN', 'Sunday_school_class|NOUN', 'female_classmate|NOUN', 'class_X|NOUN', 'school_classes|NOUN', 'stealth_class|NOUN', 'ethics_classes|NOUN', 'art_history_class|NOUN', '3rd_grade_class|NOUN', 'failed_classes|NOUN', 'only_a_few_classes|NOUN', 'classy_Reddit|PRODUCT', 'fres

Both `max_new_tokens` (=200) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[DEBUG] Generated answer: 'Greek success' for question: 'Greek success was also what by Greek domination of the Christian Orthodox church?'
'Greek_success|n' not found in the model. Exploring similar available keys...
Available keys related to 'Greek success': ['Greek_texts|NOUN', 'Greek_girl|NOUN', 'successful_doctor|NOUN', 'past_successes|NOUN', 'Greek_guy|NOUN', '^^not_^^guarantee_^^a_^^successful_^^swap|NOUN', 'Greek_state|NOUN', 'successive_years|DATE', 'moderate_success|NOUN', 'massive_success|NOUN', 'too_much_success|NOUN', 'successful_author|NOUN', 'considerable_success|NOUN', 'success|NOUN', 'first_success|NOUN', 'Greektown|PERSON', 'Greek_myths|NOUN', 'successions|NOUN', 'very_successful_people|NOUN', 'successful_defenses|NOUN', 'successful_streamer|NOUN', 'Greek_gods|NOUN', 'Greek_voters|NOUN', 'successful_business_owners|NOUN', 'Greek_origin|NOUN', 'successful_hit|NOUN', 'unsuccessfully|ADV', 'successful_attempts|NOUN', 'business_success|NOUN', 'successful_franchise|NOUN', 